In [2]:
import numpy as np
import emcee
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import interpolate
from scipy import stats
import healpy as hp  # NOTE: you can neglect warning about version mismatch in CFITSIO if necessary 

N_side = 2048      # Both maps will have this N_side. Healpix map has 12 * N_side**2 pixels

from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import units as u

import pandas as pd

import emcee

import matplotlib.pyplot as plt
import os, sys, shutil
import random
import sklearn.metrics as slm

import corner
from scipy.interpolate import interp1d
%matplotlib inline

pkdir = "/pscratch/sd/s/sbrisin/boss_cib_cmass/cibcmass/hmvec"
sys.path.insert(0, pkdir)
# Import hmvec as hm
from hmvec import hmvec as hm

# this part of the notebook prepares the data

In [3]:
CIB_map = hp.read_map('/Users/sophiarisin/Dropbox/Mac (2)/Desktop/research/cmb/catelogues/COM_CompMap_CIB-GNILC-F545_2048_R2.00.fits')        # read CIB map
CIB_mask = hp.read_map('/Users/sophiarisin/Dropbox/Mac (2)/Downloads/cib_mask.fits')                                    # read CIB mask

CIB_masked = hp.ma(CIB_map)
CIB_masked.mask = np.logical_not(CIB_mask)
hp.mollview(CIB_masked)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sophiarisin/Dropbox/Mac (2)/Desktop/research/cmb/catelogues/COM_CompMap_CIB-GNILC-F545_2048_R2.00.fits'

In [ ]:

from astropy.io import fits
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

def read_nobs_pyfits(filename):
    with fits.open(filename, memmap=True) as hdul:
        data = (hdul[1].data)
        return np.shape(data)[0], hdul[1].columns.names

def read_test_pyfits(filename, colname):
    with fits.open(filename, memmap=True) as hdul:
        data = (hdul[1].data[colname])
        return data.copy()

def get_BOSS_data(gal):
    nObs, cols = read_nobs_pyfits(gal)
    colnames = [x for x in cols if x in ['ID', 'RA', 'DEC', 'Z', 'NZ', 'BOSS_SPECOBJ_ID',
                                         'BOSS_TARGET1', 'BOSS_TARGET2', 'EBOSS_TARGET0', 'ZOFFSET', 'TARGETOBJID',
                                         'OBJID', 'PLUG_RA', 'PLUG_DEC', 'Z']]
    ncols = len(colnames)
    myGalaxy = pd.DataFrame(data=np.zeros([nObs, ncols]), columns=colnames)
    for rowname in myGalaxy.columns:
        myGalaxy[rowname] = read_test_pyfits(gal, rowname).byteswap().newbyteorder()
    print(myGalaxy.columns)
    myGalaxy = myGalaxy.sort_values(by=['Z'])
    return myGalaxy



boss_catalog_path_s = '/Users/sophiarisin/Dropbox/Mac (2)/Desktop/research/cmb/catelogues/galaxy_DR12v5_CMASS_South.fits'
boss_catalog_path_n = '/Users/sophiarisin/Dropbox/Mac (2)/Desktop/research/cmb/catelogues/galaxy_DR12v5_CMASS_North.fits'
# Load the relevant entries of the catalog (we will be mostly concerned with RA, DEC, and redshift)
boss_catalog_s = get_BOSS_data(boss_catalog_path_s)
boss_catalog_n = get_BOSS_data(boss_catalog_path_n)

# Load the relevant entries of the catalog (we will be mostly concerned with RA, DEC, and redshift)
datas = get_BOSS_data(boss_catalog_path_s)
datan = get_BOSS_data(boss_catalog_path_n)
datas = pd.DataFrame(datas)
datasmodd = pd.DataFrame(datas[(datas.Z > .4) * (datas.Z < .5)])
datanmodd = pd.DataFrame(datan[(datan.Z > .4) * (datan.Z < .5)])

modtot = pd.concat([datanmodd,datasmodd])
modtot
#datas = np.where([(datas.Z > .4) * (datas.Z < .5)])
#datan= get_BOSS_data(boss_catalog_path_n)


In [ ]:
nside = 2048

npix = hp.nside2npix(nside)
# Go from HEALPix coordinates to indices, implicitly converting to galactic coordinates
c_icrs_s = SkyCoord(ra=modtot['RA']*u.degree, dec=modtot['DEC']*u.degree, frame='icrs')
indices = hp.ang2pix(nside, c_icrs_s.galactic.l.value, c_icrs_s.galactic.b.value, lonlat=True)
#c_icrs_n = SkyCoord(ra=datan['RA']*u.degree, dec=datan['DEC']*u.degree, frame='icrs')
#indices = hp.ang2pix(nside, c_icrs_n.galactic.l.value, c_icrs_n.galactic.b.value, lonlat=True)

# Initate the map and fill it with the values
gal_hpmap = np.zeros(npix, dtype=np.float)
# Assign sources to pixels
np.add.at(gal_hpmap, indices, 1)

print(np.sum(gal_hpmap))

def get_dn_from_map(gal_hpmap):
    ''' Convert a galaxy number map obtained using get_map_from_catalog() to one of fractional
        overdensity. Return also the mask
    ''' 
    #Total number counts. Should convert to delta_n/n. Make a mask first!
    gal_downgraded = hp.ud_grade(gal_hpmap, 64)
    gal_mask = np.zeros(len(gal_downgraded))
    gal_mask[(gal_downgraded > 0)] = 1
    nside = hp.npix2nside(len(gal_hpmap))
    gal_mask = hp.ud_grade(gal_mask, nside) # bring it back to the original resolution
    
    # Convert to delta_n / n (fractional fluctuations: this is what we can predict from theory)
    mean_gal = np.sum(gal_hpmap * gal_mask) / np.sum(gal_mask)
    gal_masked_dn = (gal_hpmap * gal_mask) / mean_gal - 1.
    gal_masked_dn = gal_mask * gal_masked_dn
    return gal_masked_dn, gal_mask


gal_dn, gal_mask = get_dn_from_map(gal_hpmap)

hp.mollview(gal_dn)

gal_dn.max()
print(npix)

In [ ]:
CMASS_map = gal_hpmap # mask should be made from regular density 
CMASS_downgraded = hp.ud_grade(CMASS_map, 64)
CMASS_mask = np.zeros(len(CMASS_downgraded))
CMASS_mask[(CMASS_downgraded > 0)] = 1
CMASS_mask = hp.ud_grade(CMASS_mask, N_side) # bring it back to the original resolution
hp.mollview(CMASS_mask,)
coord= ['G','C']
# Convert to a masked array for better plotting
CMASS_masked_dn = hp.ma(gal_dn)
CMASS_masked_dn.mask = np.logical_not(CMASS_mask)
hp.mollview(hp.ud_grade(CMASS_masked_dn,64), title =' .6 < z <.8')

In [ ]:
# The "total" mask is product of PLANCK and CMASS masks
mask_tot = CMASS_mask * CIB_mask
fsky = np.sum(mask_tot) / len(mask_tot) # fsky for the cross-correlation
fsky_CIB = np.sum(CIB_mask) / len(CIB_mask)
fsky_CMASS = np.sum(CMASS_mask) / len(CMASS_mask)
print("f_sky cross = " + str(fsky))
print("f_sky CIB = " + str(fsky_CIB))
print("f_sky CMASS = " + str(fsky_CMASS))

In [ ]:
Cl_cross = hp.anafast(CIB_masked, map2 = CMASS_masked_dn, lmax = 2000)
ls = np.arange(len(Cl_cross))
pixwinf = hp.pixwin(N_side)[0:len(Cl_cross)]
# further divide by one power of beam
Cl_cross = Cl_cross / (pixwinf **2)                # Remove pixel window function
Cl_cross = Cl_cross / fsky                         # Undo the dilution caused by having observed only part of the sky

In [ ]:
# Same as before!
Cl_CIB = hp.anafast(CIB_masked, lmax = 2000)
Cl_CMASS = hp.anafast(CMASS_masked_dn, lmax = 2000)
Cl_CIB = Cl_CIB / (pixwinf **2)
Cl_CMASS = Cl_CMASS / (pixwinf **2)
Cl_CIB = Cl_CIB / fsky_CIB
Cl_CMASS = Cl_CMASS / fsky_CMASS

In [ ]:
corrbins = pd.DataFrame({'l': ls, 'cl': Cl_cross, 'cib': Cl_CIB, 'cmass': Cl_CMASS})

#size it up 
things = (pd.DataFrame(corrbins[(corrbins.l > 100) * (corrbins.l < 1000)]))
things = things.dropna()
df = things.reset_index(drop=True)
dfo = abs(df)
dfo = np.log(dfo)


l = df['l']
llog = dfo['l']
cllog = dfo['cl']
#llog = np.log(l)
cl = df['cl']
cib = df['cib']
cmass = df['cmass']

zs = modtot['Z']
gdndz = np.array(plt.hist(zs))

In [ ]:
# this bins the data that we use for running our mcmc in the end 
Nbins = 5
lmin = 300
lmax = 1000

bins = np.round(np.linspace(lmin, lmax, Nbins+1))   # Bin edges
bins = bins.astype(int)
lcenterbin = np.zeros(len(bins)-1)
binned_cross = np.zeros(len(bins)-1)
binned_CIB = np.zeros(len(bins)-1)
binned_CMASS = np.zeros(len(bins)-1)

for k in range(0, len(bins)-1):  
    lmaxvec = np.arange(bins[k], bins[k+1], 1)
    lcenterbin[k] = np.round(0.5 * (bins[k] + bins[k+1]))   # bin center
    for l in lmaxvec:
        binned_cross[k] += Cl_cross[l]
    binned_cross[k] = binned_cross[k] / len(lmaxvec)

for k in range(0, len(bins)-1): 
    lmaxvec = np.arange(bins[k], bins[k+1], 1)
    for l in lmaxvec:
        binned_CIB[k] += Cl_CIB[l]
    binned_CIB[k] = binned_CIB[k] / len(lmaxvec)

for k in range(0, len(bins)-1): 
    lmaxvec = np.arange(bins[k], bins[k+1], 1)
    for l in lmaxvec:
        binned_CMASS[k] += Cl_CMASS[l]
    binned_CMASS[k] = binned_CMASS[k] / len(lmaxvec)
    
    sigmavecth = np.zeros(len(bins)-1)
for k in range(0, len(bins)-1):
    lmaxvec = np.arange(bins[k], bins[k+1], 1)
    for l in lmaxvec:
        sigmavecth[k] += fsky * (2. * l + 1.) / (Cl_CIB[l] * Cl_CMASS[l] + Cl_cross[l]**2)
    sigmavecth[k] = 1. / sigmavecth[k]
sigmavecth = np.sqrt(sigmavecth)

this part of the notebook gets the covariance

In [ ]:
def var_gg(cgg,l)
    f_sky_cross = 0.249634150739921
    f_sky_CIB = 0.5787552197774252
    f_sky_CMASS = 0.2552083333333333
    return ((2*cgg)/((2*l+1)*(f_sky_CMASS)))
def var_ig(cii,cgg,cig,l)
    f_sky_cross = 0.249634150739921
    f_sky_CIB = 0.5787552197774252
    f_sky_CMASS = 0.2552083333333333
    return (cgg*cii+cig**2)/((2*l+1)*f_sky_cross)

def cov_cgg_cgi(cii,cgg,l)
    f_sky_cross = 0.249634150739921
    f_sky_CIB = 0.5787552197774252
    f_sky_CMASS = 0.2552083333333333
    return (2*cgg*cig)/((2*l+1)*f_sky_CMASS)

In [ ]:
a=0
b = 0
c = 0
jp =0 
d = 0
e = 0
f = 0
g = 0
p = 0
r= 0
q= 0
k = 0
ells = np.arange(0,701)
for i in ells:
    if 0 <= i < 140:
        a_l = var_gg[i]
        b_l = var_ig[i]
        a += 1/a_l
        a = 1/a
        b += 1/b_l
        b = 1/b
    elif 140 <= i < 280:
        j_l = var_gg[i]
        d_l = var_ig[i]
        jj = cov_cgg_cgi[i]
        jp += 1/j_l
        jp = 1/jp
        d += 1/d_l
        d = 1/d
        e += (jj/(j_l*d_l))
        fc = (jp)*(d)*(e)
    elif 280 <= i < 420:
        f_l = var_gg[i]
        g_l = var_ig[i]
        h = cov_cgg_cgi[i]
        f += 1/f_l
        f = 1/f
        g += 1/g_l
        g = 1/g
        h += (h/(f_l*g_l))
        aac = (f)*(g)*(h)
    elif 420 <= i < 560:
        p_l = var_gg[i]
        k_l = var_ig[i]
        n = cov_cgg_cgi[i]
        p += 1/p_l
        p = 1/p
        k += 1/k_l
        k = 1/k
        n += (n/(p_l*k_l))
        hc = (n)*(p)*(k)
    elif 560 <=i < 700:
        q_l = var_gg[i]
        r_l = var_ig[i]
        s = cov_cgg_cgi[i]
        q += 1/q_l
        q = 1/q
        r += 1/r_l
        r = 1/r
        s += (s/(q_l*r_l))
        sc = (s)*(q)*(r)
binnedvar = np.array([b,d,g,k,r])

now ml part

In [ ]:
def theory(theta,l_min,l_max,bins):
    '''
    x is Log M_eff
    y is L0
    this function takes in theta and returns an array of the C_ls for galaxy and the CIB
    at the center of the given redshift bins
    the number of elements must match the number of elements in the data
    this function is a big big big function but takes in, l range, theta, number of bins
    and returns the c_l gg, cl gi, cov, cov inverse, yea
    
    gdndz, and zs come from the eBOSS galaxy catelogue
    requires: hcos, hmvec
    be sure to run this per each redshift bin
    this is specific the version to run for the mcmc <3
    '''

    ells = np.arange(3000)
    cib_freq = 545 * 1e9


    zs = np.linspace(.4, .5, 10) # make this variable
    ms = np.geomspace(1e10, 1e17, 20) # revisit the webb sky, 
    #lower mass limit and see if the disagreement goes away 
    ks = np.geomspace(1e-4, 100, 100)

    hcos = hm.HaloModel(zs, ks, ms=ms)
    hcos.set_cibParams('vierro')

        # a and b are the z range
    hcos.cib_params['alpha'] = 0.2
    hcos.cib_params['beta'] = 1.6
    hcos.cib_params['gamma'] = 1.7  # not in Viero, so using Planck13
    hcos.cib_params['delta'] = 2.4
    hcos.cib_params['Td_o'] = 20.7
    x = hcos.cib_params['logM_eff']
    hcos.cib_params['var'] = 0.3
    y = hcos.cib_params['L_o'] 
    hcos.add_hod(name="CMASS", mthresh=10**12 + zs*0.)
        
    # this is for the c_gg, make these inputs
    gdndz = gdndz[0]
    zs = gdndz[1]

    Pgg_1h = hcos.get_power_1halo('CMASS', 'CMASS', nu_obs=np.array([cib_freq]))
    Pgg_2h = hcos.get_power_2halo('CMASS', 'CMASS', nu_obs=np.array([cib_freq]))

    Cl_gg_1h = hcos.C_gg(ells, hcos.zs, hcos.ks, Pgg_1h, gzs=zs, gdndz=gdndz)
    Cl_gg_2h = hcos.C_gg(ells, hcos.zs, hcos.ks, Pgg_2h, gzs=zs, gdndz=gdndz)
    tot_gg = Cl_gg_1h + Cl_gg_2h # add the halos together
    # for the cl gi
    PgI_1h = hcos.get_power_1halo('CMASS', 'cib', nu_obs=np.array([cib_freq]))
    PgI_2h = hcos.get_power_2halo('CMASS', 'cib', nu_obs=np.array([cib_freq]))

    Cl_gI_1h = hcos.C_gI(ells, hcos.zs, hcos.ks, PgI_1h, gzs=zs, gdndz= gdndz)
    Cl_gI_2h = hcos.C_gI(ells, hcos.zs, hcos.ks, PgI_2h, gzs=zs, gdndz= gdndz)
    
    # need to end up with one vector thats c_gi for each l bin, c_gg for each l bin
    tot_gi = Cl_gI_1h + Cl_gI_2h
    #370.0,510.0,650.0,790.0, 930.0
    baba = np.array([tot_gi[370],tot_gi[510],tot_gi[650],tot_gi[790],tot_gi[930]])
        
    return baba

In [ ]:
dat = np.array([1.506013e-08, 1.593685e-08,1.071126e-08, 1.097310e-08, 5.175340e-09]) # this is the data
cov = np.diag(binnedvar)

# this is your covariance matrix
cinv = np.linalg.inv(cov) # inverse covariance matrix

In [ ]:
def model(theta, dat = dat):
    x,y =  theta 
    return theory(theta,300,1000,5)
def lnprior(theta):
    x,y =  theta 
    return 0.0
def lnlike(theta,cinv,cov,dat = dat):
    thy = theory(theta,300,1000,5) # this works dimensionally
    diff = thy-dat
    return -np.dot(diff,np.dot(cinv,diff))/2
def lnprob(theta):
    lp = lnlike(theta,cinv,cov)
    print(lp)
    if lp == -np.inf:
        return -np.inf
    #print(lp + lnprior(theta))
    return lp + lnprior(theta)

In [ ]:
def main(p0,nwalkers,niter,ndim,lnprob,data):
    sampler =  emcee.EnsembleSampler(nwalkers,ndim, lnprob, args = data)
    print('running baybee..')
    p0, _, _, _= sampler.run_mcmc(p0,1)
    sampler.reset()
    
    print( 'running prod')
    pos, prob, state, _ =  sampler.run_mcmc(p0,niter)
    print('done!')
    return sampler, pos, prob, state

In [ ]:
data = (dat,cov,cinv)
nwalkers = 10
niter = 2
# set n walkers
# set niter 
initial = np.array([2e-7,12.3]) # these come from the initial vierro params
ndim = 2 # only 2 params so 2 dim
p0 = [np.array(initial) + np.random.randn(ndim) for i in range(nwalkers)]

In [ ]:
sampler, pos, prob, state = main(p0,nwalkers,niter,ndim,lnlike,data)
samples = sampler.flatchain

In [ ]:
labels = ['L_o', 'Log_Meff']
fig = corner.corner(samples,show_titles=True,labels=labels,plot_datapoints=True,quantiles=[0.16, 0.5, 0.84])